In [1]:
import json
import pandas as pd

import portrisk

from deribit import get_order_book

pd.set_option('display.max_rows', None)  # Set max_rows to None to display all rows
pd.set_option('display.max_columns', None)  # Set max_columns to None to display all columns


In [3]:
# Load stress parameters
file_path = './parameters/'

with open(file_path + 'spot_vol_shocks.json', 'r') as f:
    spot_vol_shocks_param = json.load(f)

crypto_parameters = portrisk.core.utils.CryptoParameters(spot_vol_shocks_param)

with open(file_path + 'vol_surface_shocks.json', 'r') as f:
    vol_surface_shocks_param = json.load(f)

vol_surface_shocks_parameters = portrisk.core.utils.VolSurfaceParameters(vol_surface_shocks_param)


In [34]:
# load position data
df = pd.read_excel('positions.xlsx')

# Splitting the 'instrument' column by "-"
df[['underlying','expiry','strike','put_call']] = df['instrument'].str.split('-', expand=True)
df['strike'] = df['strike'].astype(float)
df['put_call'] = df['put_call'].map({'P':'put','C':'call'})
df['expiry'] = pd.to_datetime(df['expiry'], format='%d%b%y')

# Calculate time to expiry
valuation_day = pd.to_datetime('2024-08-05')
df['time_to_expiry'] = (df['expiry'] - valuation_day).dt.days / 365

# get market data from deribit
deribit_res = get_order_book(instruments=df['instrument'].to_list())
df = pd.merge(df, deribit_res,how='left',left_on='instrument',right_on='instrument_name')


# calculate input for BSM		
df['cost_of_carry_rate'] = 'default'
df['multiplier'] = 1
df['rate'] = 0.03
df['vol'] = df['mark_iv'] / 100
df['spot'] = df['index_price'].values[0]

df['PositionVega'] = df['vega'] * df['quantity']
df['Expiry'] = df['expiry']

# calculate atm_ivol 

# atm_ivol_map = {}
# for u in df['underlying'].to_list():
#     tem = df[df['underlying'].isin([u])].copy()
#     # Find the closest value in column strike to the spot
#     closest_value = tem['strike'].iloc[(tem['strike'] - tem['spot'].values[0]).abs().argsort()[0]]
#     # Retrieve the corresponding value from column vol
#     atm_ivol_map[u] = tem.loc[df['strike'] == closest_value, 'vol'].values[0]

# df['atm_ivol'] = df['underlying'].map(atm_ivol_map)
# Function to get ATM implied volatility for a specific group

def get_atm_iv(group):
    group['diff'] = (group['strike'] - group['spot']).abs()
    atm_index = group['diff'].idxmin()
    atm_iv = group.at[atm_index, 'vol']
    group['atm_ivol'] = atm_iv
    return group

# Apply the function to each group
df = df.groupby('expiry',as_index=False, group_keys=False).apply(get_atm_iv).reset_index(drop=True)

# Drop the temporary 'diff' column
df.drop(columns=['diff'], inplace=True)


## Spot Vol Shocks

In [5]:
spot_vol_shocks = portrisk.crypto.CryptoSpotVolShocks(crypto_parameters)

In [8]:
post_stress, col = spot_vol_shocks.apply_spot_vol_shocks(df)

In [23]:
portrisk.core.black_scholes.bs_pricing(strike=50000, time_to_expiry=0.232877, spot=54681.59, rate=0.03, vol=0.5775, put_call='put')
portrisk.core.black_scholes.calc_vega(strike=50000, time_to_expiry=0.232877, spot=54681.59, rate=0.03, vol=0.5775, put_call='put')

# np.float64(3606.873391368419)
# np.float64(3700.518323796012)

np.float64(93.56524889234046)

In [41]:
post_stress

,instrument,quantity,underlying,expiry,strike,put_call,time_to_expiry,underlying_price,timestamp,mark_iv,instrument_name,index_price,rho,theta,vega,gamma,delta,bid_iv,ask_iv,best_bid_price,best_ask_price,cost_of_carry_rate,multiplier,rate,vol,spot,PositionVega,Expiry,spot_shock,vol_shock,scenario_1,scenario_2
0,BTC-25OCT24-50000-P,-25.0,BTC,2024-10-25,50000.0,put,0.221918,56917.6415,1722909201139,58.61,BTC-25OCT24-50000-P,55830.18,-40.58584,-32.30232,88.45808,0.00002,4.008412e+05,57.97,59.25,0.0520,0.0540,default,1,0.03,0.5861,55830.18,-2211.452000,2024-10-25,0.15,0,32336.680345,43632.307063
1,BTC-25OCT24-84000-C,0.0,BTC,2024-10-25,84000.0,call,0.221918,56910.8831,1722909201670,64.35,BTC-25OCT24-84000-C,55824.65,13.81209,-22.30537,55.63322,0.00001,0.000000e+00,63.41,64.95,0.0160,0.0175,default,1,0.03,0.6435,55830.18,0.000000,2024-10-25,0.15,0,0.000000,0.000000
2,BTC-27DEC24-105000-C,-8.8,BTC,2024-12-27,105000.0,call,0.394521,57895.3800,1722909202219,68.53,BTC-27DEC24-105000-C,55820.34,22.52019,-17.41286,72.80398,0.00001,-5.421136e+04,67.74,68.94,0.0205,0.0220,default,1,0.03,0.6853,55830.18,-640.675024,2024-12-27,0.15,0,-6559.130476,-10755.863307
3,BTC-27DEC24-110000-C,-28.9,BTC,2024-12-27,110000.0,call,0.394521,57895.3800,1722909202617,69.65,BTC-27DEC24-110000-C,55820.34,19.68645,-16.04940,66.01927,0.00001,-1.549794e+05,68.75,70.08,0.0175,0.0195,default,1,0.03,0.6965,55830.18,-1907.956903,2024-12-27,0.15,0,-18838.389584,-30968.634250
4,BTC-27DEC24-115000-C,-1.7,BTC,2024-12-27,115000.0,call,0.394521,57897.2500,1722909203254,70.77,BTC-27DEC24-115000-C,55822.02,17.33582,-14.83783,60.06649,0.00001,-8.000408e+03,69.68,71.61,0.0150,0.0170,default,1,0.03,0.7077,55830.18,-102.113033,2024-12-27,0.15,0,-976.250083,-1608.239119
5,BTC-27DEC24-120000-C,95.0,BTC,2024-12-27,120000.0,call,0.394521,57901.0200,1722909204305,71.70,BTC-27DEC24-120000-C,55822.02,15.24825,-13.63429,54.48133,0.00001,3.917676e+05,71.17,72.24,0.0135,0.0145,default,1,0.03,0.7170,55830.18,5175.726350,2024-12-27,0.15,0,48006.334944,79261.281440
6,BTC-27DEC24-43000-P,31.4,BTC,2024-12-27,43000.0,put,0.394521,57900.7600,1722909205312,60.68,BTC-27DEC24-43000-P,55825.36,-46.70101,-19.09524,90.16274,0.00001,-3.195464e+05,60.25,61.21,0.0395,0.0410,default,1,0.03,0.6068,55830.18,2831.110036,2024-12-27,0.15,0,-26516.598708,-36352.445188
7,BTC-27DEC24-44000-P,20.6,BTC,2024-12-27,44000.0,put,0.394521,57893.7100,1722909206214,60.50,BTC-27DEC24-44000-P,55818.30,-51.19913,-20.12869,95.32817,0.00001,-2.280412e+05,59.98,60.89,0.0440,0.0455,default,1,0.03,0.6050,55830.18,1963.760302,2024-12-27,0.15,0,-19018.948216,-26130.320710
8,BTC-27DEC24-45000-P,14.0,BTC,2024-12-27,45000.0,put,0.394521,57892.7700,1722909207222,60.33,BTC-27DEC24-45000-P,55817.35,-55.89478,-21.13015,100.34708,0.00001,-1.679102e+05,59.83,60.69,0.0490,0.0505,default,1,0.03,0.6033,55830.18,1404.859120,2024-12-27,0.15,0,-14073.714244,-19377.612669
9,BTC-27DEC24-47000-P,-9.0,BTC,2024-12-27,47000.0,put,0.394521,57898.8400,1722909208433,60.12,BTC-27DEC24-47000-P,55817.35,-65.93148,-23.05809,109.88260,0.00001,1.253641e+05,59.81,60.60,0.0605,0.0620,default,1,0.03,0.6012,55830.18,-988.943400,2024-12-27,0.15,0,10613.777021,14678.064812


## Vol Surface Shocks

In [6]:
vol_surface_shocks = portrisk.crypto.CryptoVolSurfaceShocks(vol_surface_shocks_parameters)

In [7]:
vol_surface_shocks.run(df)

[{'product': 'BTC',
  'measure': 'Concentration',
  'value': np.float64(-108067.96142879587),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'BTC',
  'measure': 'TermStructure',
  'value': np.float64(-173748.14657475732),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'BTC',
  'measure': 'Skew',
  'value': np.float64(-18.707505459235108),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'BTC',
  'measure': 'BidAsk',
  'value': np.float64(-8675.83704731747),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'BTC',
  'measure': 'Sum',
  'value': np.float64(-290510.6525563299),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'Sum',
  'measure': 'Concentration',
  'value': -108067.96142879587,
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'Sum',
  'measure': 'TermStructure',
  'value': -173748.14657475732,
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'

In [ ]:

# df[['instrument','expiry','atm_ivol','vol']]

## Ivol Timing

In [2]:
df = pd.read_excel('ivol_timing.xlsx')
df['time'] = pd.to_datetime(df['timestamp'], unit='ms')

In [28]:
# Volatility Index

from datetime import datetime, timedelta
import requests

ccy='BTC'
end_timestamp = int(datetime.now().timestamp() * 1000)
time_24_hours_ago = datetime.now() - timedelta(hours=24)
start_timestamp = int(time_24_hours_ago.timestamp() * 1000)
url = f"https://test.deribit.com/api/v2/public/get_volatility_index_data"

r = requests.get(url, params = {'currency': ccy,
                                'end_timestamp': end_timestamp,
                                'start_timestamp': start_timestamp,
                                'resolution':3600})

df_ivol_index = pd.DataFrame(r.json()['result']['data']).rename(columns={0:'timestamp',1:'open',2:'high',3:'low',4:'close'})
df_ivol_index['time'] = pd.to_datetime(df_ivol_index['timestamp'], unit='ms')
df_ivol_index['ivol_index'] = df_ivol_index['close']


In [29]:
import plotly.graph_objects as go

# Create separate traces for buy and sell
buy_trace = go.Scatter(x=df[df['direction'] == ' buy']['time'], y=df[df['direction'] == ' buy']['iv'],
                       mode='markers', marker=dict(color='blue', size=5), name='Buy')
sell_trace = go.Scatter(x=df[df['direction'] == ' sell']['time'], y=df[df['direction'] == ' sell']['iv'],
                        mode='markers', marker=dict(color='red', size=5), name='Sell')
# Line trace for ivol data
line_trace = go.Scatter(
    x=df_ivol_index['time'],
    y=df_ivol_index['ivol_index'],
    mode='lines',
    line=dict(color='green'),
    name='Ivol Index'
)

# Create figure with both traces
fig = go.Figure([buy_trace, sell_trace,line_trace])

# Update layout
fig.update_layout(title='IV vs Time',
                  xaxis_title='Time',
                  yaxis_title='IV')

# Show plot
fig.show()